# Get Drug Case Inclusion

In [1]:
library(tidyverse)
library(bigrquery)

# This query represents dataset "asthma_steroids" for domain "drug" and was generated for All of Us Controlled Tier Dataset v7
dataset_45376339_drug_sql <- paste("
    SELECT
        d_exposure.person_id,
        d_exposure.drug_concept_id,
        d_standard_concept.concept_name as standard_concept_name,
        d_standard_concept.concept_code as standard_concept_code,
        d_standard_concept.vocabulary_id as standard_vocabulary 
    FROM
        ( SELECT
            * 
        FROM
            `drug_exposure` d_exposure 
        WHERE
            (
                drug_concept_id IN (SELECT
                    DISTINCT ca.descendant_id 
                FROM
                    `cb_criteria_ancestor` ca 
                JOIN
                    (SELECT
                        DISTINCT c.concept_id       
                    FROM
                        `cb_criteria` c       
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id             
                        FROM
                            `cb_criteria` cr             
                        WHERE
                            concept_id IN (1110942, 1115572, 1149380, 1196514, 1506270, 1518254, 1550557, 1551099, 36883745, 902938, 903963, 905233, 939259)             
                            AND full_text LIKE '%_rank1]%'       ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) b 
                        ON (ca.ancestor_id = b.concept_id)))  
                    AND (d_exposure.PERSON_ID IN (SELECT
                        distinct person_id  
                FROM
                    `cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `cb_search_person` p 
                    WHERE
                        has_ehr_data = 1 ) )
            )) d_exposure 
    LEFT JOIN
        `concept` d_standard_concept 
            ON d_exposure.drug_concept_id = d_standard_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
drug_45376339_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "drug_45376339",
  "drug_45376339_*.csv")
message(str_glue('The data will be written to {drug_45376339_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_45376339_drug_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  drug_45376339_path,
  destination_format = "CSV")

# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {drug_45376339_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
steroids_df <- read_bq_export_from_workspace_bucket(drug_45376339_path)

unique(steroids_df$standard_concept_name)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
The data will be written to gs://fc-secure-672eeb92-4859-4ed9-9f59-d4349f3534a0/bq_exports/micah_hysong@researchallofus.org/20250825/drug_45376339/drug_45376339_*.csv. Use this path when reading the data into your notebooks in the future.

Loading gs://fc-secure-672eeb92-4859-4ed9-9f59-d4349f3534a0/bq_exports/micah_hysong@researchallofus.org/20250825/drug_45376339/drug_45376339_000000000000.csv.



[1] "prednisone 10 MG Oral Tablet [Orasone]"                                                                                                     
  [2] "prednisolone acetate 10 MG/ML Ophthalmic Suspension [Pred Forte]"                                                                           
  [3] "nystatin 100 UNT/MG / triamcinolone acetonide 0.001 MG/MG Topical Ointment [Myocidin]"                                                      
  [4] "prednisone 20 MG Oral Tablet [Predone]"                                                                                                     
  [5] "budesonide 0.08 MG/ACTUAT / formoterol fumarate 0.0045 MG/ACTUAT Metered Dose Inhaler [Symbicort]"                                          
  [6] "60 ACTUAT formoterol fumarate 0.005 MG/ACTUAT / mometasone furoate 0.2 MG/ACTUAT Metered Dose Inhaler [Dulera]"                             
  [7] "prednisone 5 MG Oral Tablet [Deltasone]"                                                                                                    
  [8] "60 ACTUAT flunisolide 0.08 MG/ACTUAT Metered Dose Inhaler [Aerospan]"                                                                       
  [9] "triamcinolone acetonide 40 MG/ML Injectable Suspension [Triamcot]"                                                                          
 [10] "nystatin 100000 UNT/ML / triamcinolone acetonide 1 MG/ML Topical Cream [Mycogen]"                                                           
 [11] "triamcinolone acetonide 1 MG/ML Topical Cream [Kenalog]"                                                                                    
 [12] "budesonide 9 MG Extended Release Oral Capsule [Ortikos]"                                                                                    
 [13] "1 ML dexamethasone phosphate 10 MG/ML Prefilled Syringe"                                                                                    
 [14] "fluticasone propionate 0.22 MG/ACTUAT Metered Dose Inhaler"                                                                                 
 [15] "prednisolone sodium phosphate 2.5 MG/ML / sulfacetamide sodium 100 MG/ML Ophthalmic Solution [Vasocidin]"                                   
 [16] "budesonide Dry Powder Inhaler"                                                                                                              
 [17] "dexamethasone phosphate 1 MG/ML Ophthalmic Solution"                                                                                        
 [18] "14 ACTUAT fluticasone propionate 0.5 MG/ACTUAT / salmeterol 0.05 MG/ACTUAT Dry Powder Inhaler [Advair]"                                     
 [19] "14 ACTUAT fluticasone furoate 0.1 MG/ACTUAT Dry Powder Inhaler [Arnuity]"                                                                   
 [20] "budesonide 0.25 MG/ML Inhalation Suspension [Pulmicort]"                                                                                    
 [21] "{21 (prednisolone 5 MG Oral Tablet) } Pack"                                                                                                 
 [22] "120 ACTUAT flunisolide 0.08 MG/ACTUAT Metered Dose Inhaler [Aerospan]"                                                                      
 [23] "triamcinolone acetonide 0.00025 MG/MG Topical Ointment [Aristocort]"                                                                        
 [24] "dexamethasone phosphate 4 MG/ML Injection"                                                                                                  
 [25] "triamcinolone acetonide 0.25 MG/ML Topical Cream [Triacet]"                                                                                 
 [26] "triamcinolone acetonide 1 MG/ML Topical Lotion"                                                                                             
 [27] "120 ACTUAT formoterol fumarate 0.005 MG/ACTUAT / mometasone furoate 0.2 MG/ACTUAT Metered Dose Inhaler [Dulera]"                            
 [28] 

In [2]:
steroid_perscription_holders<-unique(steroids_df$person_id)
length(steroid_perscription_holders)

[1] 177584

In [3]:
library(tidyverse)
library(bigrquery)

# This query represents dataset "asthma_mild_drugs" for domain "drug" and was generated for All of Us Controlled Tier Dataset v7
dataset_97024772_drug_sql <- paste("
    SELECT
        d_exposure.person_id,
        d_exposure.drug_concept_id,
        d_standard_concept.concept_name as standard_concept_name,
        d_standard_concept.concept_code as standard_concept_code,
        d_standard_concept.vocabulary_id as standard_vocabulary 
    FROM
        ( SELECT
            * 
        FROM
            `drug_exposure` d_exposure 
        WHERE
            (
                drug_concept_id IN (SELECT
                    DISTINCT ca.descendant_id 
                FROM
                    `cb_criteria_ancestor` ca 
                JOIN
                    (SELECT
                        DISTINCT c.concept_id       
                    FROM
                        `cb_criteria` c       
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id             
                        FROM
                            `cb_criteria` cr             
                        WHERE
                            concept_id IN (1111220, 1111706, 1112921, 1114620, 1123995, 1125449, 1137529, 1147878, 1152631, 1154161, 1154343, 1192218, 1196677, 1236744, 1237049, 1563413)             
                            AND full_text LIKE '%_rank1]%'       ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) b 
                        ON (ca.ancestor_id = b.concept_id)))  
                    AND (d_exposure.PERSON_ID IN (SELECT
                        distinct person_id  
                FROM
                    `cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `cb_search_person` p 
                    WHERE
                        has_ehr_data = 1 ) )
            )) d_exposure 
    LEFT JOIN
        `concept` d_standard_concept 
            ON d_exposure.drug_concept_id = d_standard_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
drug_97024772_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "drug_97024772",
  "drug_97024772_*.csv")
message(str_glue('The data will be written to {drug_97024772_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_97024772_drug_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  drug_97024772_path,
  destination_format = "CSV")


# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {drug_97024772_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
asthma_mild_drug_df <- read_bq_export_from_workspace_bucket(drug_97024772_path)

unique(asthma_mild_drug_df$standard_concept_name)

The data will be written to gs://fc-secure-672eeb92-4859-4ed9-9f59-d4349f3534a0/bq_exports/micah_hysong@researchallofus.org/20250825/drug_97024772/drug_97024772_*.csv. Use this path when reading the data into your notebooks in the future.

Loading gs://fc-secure-672eeb92-4859-4ed9-9f59-d4349f3534a0/bq_exports/micah_hysong@researchallofus.org/20250825/drug_97024772/drug_97024772_000000000000.csv.



[1] "pirbuterol acetate 0.2 MG/ACTUAT [Maxair]"                                                                                                  
  [2] "104 ACTUAT nedocromil sodium 1.75 MG/ACTUAT Metered Dose Inhaler [Tilade]"                                                                  
  [3] "terbutaline sulfate 2.5 MG Oral Tablet [Brethine]"                                                                                          
  [4] "theophylline 600 MG"                                                                                                                        
  [5] "60 ACTUAT fluticasone propionate 0.23 MG/ACTUAT / salmeterol 0.021 MG/ACTUAT Metered Dose Inhaler"                                          
  [6] "fluticasone propionate 0.115 MG/ACTUAT / salmeterol 0.021 MG/ACTUAT Metered Dose Inhaler"                                                   
  [7] "28 ACTUAT fluticasone propionate 0.5 MG/ACTUAT / salmeterol 0.05 MG/ACTUAT Dry Powder Inhaler"                                              
  [8] "Albuterol 0.2 MG Metered Dose Inhaler [Ventolin]"                                                                                           
  [9] "80 ACTUAT pirbuterol acetate 0.2 MG/ACTUAT Metered Dose Inhaler [Maxair]"                                                                   
 [10] "budesonide 0.16 MG/ACTUAT / formoterol fumarate 0.0048 MG/ACTUAT / glycopyrrolate 0.009 MG/ACTUAT Metered Dose Inhaler"                     
 [11] "Albuterol 0.09 MG/ACTUAT Inhalant Solution"                                                                                                 
 [12] "fluticasone propionate 0.1 MG/ACTUAT / salmeterol 0.05 MG/ACTUAT Dry Powder Inhaler [Advair]"                                               
 [13] "theophylline 300 MG Extended Release Oral Tablet"                                                                                           
 [14] "80 ACTUAT albuterol 0.09 MG/ACTUAT Metered Dose Inhaler [Ventolin]"                                                                         
 [15] "theophylline 300 MG Extended Release Oral Capsule"                                                                                          
 [16] "terbutaline sulfate 5 MG Oral Tablet [Brethine]"                                                                                            
 [17] "formoterol fumarate 0.0048 MG/ACTUAT / glycopyrrolate 0.009 MG/ACTUAT Metered Dose Inhaler"                                                 
 [18] "zafirlukast 10 MG [Accolate]"                                                                                                               
 [19] "theophylline 600 MG Extended Release Oral Tablet [Uniphyl]"                                                                                 
 [20] "60 ACTUAT aclidinium bromide 0.4 MG/ACTUAT / formoterol fumarate 0.012 MG/ACTUAT Dry Powder Inhaler [Duaklir]"                              
 [21] "60 ACTUAT salmeterol 0.05 MG/ACTUAT Dry Powder Inhaler"                                                                                     
 [22] "theophylline 300 MG Extended Release Oral Tablet [Pliva brand of Theophylline]"                                                             
 [23] "80 ACTUAT levalbuterol 0.045 MG/ACTUAT Metered Dose Inhaler"                                                                                
 [24] "salmeterol 0.021 MG/ACTUAT Metered Dose Inhaler"                                                                                            
 [25] "120 ACTUAT formoterol fumarate 0.005 MG/ACTUAT / mometasone furoate 0.05 MG/ACTUAT Metered Dose Inhaler [Dulera]"                           
 [26] "albuterol Oral Tablet [Proventil]"                                                                                                          
 [27] "zileuton"                                                                                                                                   
 [28] 

In [4]:
mild_asthma_drug_perscription_holders<-unique(asthma_mild_drug_df$person_id)
length(mild_asthma_drug_perscription_holders)

[1] 106774

In [5]:
asthma_drug_cases <- unique(c(steroid_perscription_holders, mild_asthma_drug_perscription_holders))
length(asthma_drug_cases)

[1] 190174

# Get those with Asthma Diagnosis

In [6]:
library(tidyverse)
library(bigrquery)

# This query represents dataset "asthma_snomed" for domain "condition" and was generated for All of Us Controlled Tier Dataset v7
dataset_27096384_condition_sql <- paste("
    SELECT
        c_occurrence.person_id,
        c_occurrence.condition_concept_id,
        c_standard_concept.concept_name as standard_concept_name,
        c_standard_concept.concept_code as standard_concept_code,
        c_standard_concept.vocabulary_id as standard_vocabulary,
        c_occurrence.condition_start_datetime 
    FROM
        ( SELECT
            * 
        FROM
            `condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `cb_criteria` cr       
                    WHERE
                        concept_id IN (317009)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )  
            AND (
                c_occurrence.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `cb_search_person` p 
                    WHERE
                        has_ehr_data = 1 ) )
            )) c_occurrence 
    LEFT JOIN
        `concept` c_standard_concept 
            ON c_occurrence.condition_concept_id = c_standard_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
condition_27096384_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "condition_27096384",
  "condition_27096384_*.csv")
message(str_glue('The data will be written to {condition_27096384_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_27096384_condition_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  condition_27096384_path,
  destination_format = "CSV")


# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {condition_27096384_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
asthma_condition_df <- read_bq_export_from_workspace_bucket(condition_27096384_path)

unique(asthma_condition_df$standard_concept_name)


The data will be written to gs://fc-secure-672eeb92-4859-4ed9-9f59-d4349f3534a0/bq_exports/micah_hysong@researchallofus.org/20250825/condition_27096384/condition_27096384_*.csv. Use this path when reading the data into your notebooks in the future.

Loading gs://fc-secure-672eeb92-4859-4ed9-9f59-d4349f3534a0/bq_exports/micah_hysong@researchallofus.org/20250825/condition_27096384/condition_27096384_000000000000.csv.



[1] "Acute exacerbation of chronic obstructive airways disease with asthma"    
 [2] "Aspirin exacerbated respiratory disease"                                  
 [3] "Cough variant asthma"                                                     
 [4] "Aspirin-induced asthma"                                                   
 [5] "Intermittent asthma"                                                      
 [6] "Acute severe exacerbation of moderate persistent allergic asthma"         
 [7] "Uncomplicated allergic asthma"                                            
 [8] "Persistent asthma, well controlled"                                       
 [9] "Chronic asthmatic bronchitis"                                             
[10] "Late onset asthma"                                                        
[11] "Exacerbation of allergic asthma due to infection"                         
[12] "Acute exacerbation of intrinsic asthma"                                   
[13] "Acute severe exacerbation of asthma"                                      
[14] "Asthma in pregnancy"                                                      
[15] "Exacerbation of intermittent asthma"                                      
[16] "Moderate persistent asthma"                                               
[17] "Acute severe exacerbation of immunoglobin E-mediated allergic asthma"     
[18] "IgE-mediated allergic asthma"                                             
[19] "Intrinsic asthma"                                                         
[20] "Allergic bronchopulmonary aspergillosis"                                  
[21] "Severe persistent asthma"                                                 
[22] "Asthma without status asthmaticus"                                        
[23] "Acute severe exacerbation of severe persistent asthma"                    
[24] "Acute exacerbation of chronic asthmatic bronchitis"                       
[25] "Severe asthma"                                                            
[26] "Hay fever with asthma"                                                    
[27] "Uncomplicated severe persistent asthma"                                   
[28] "Acute severe refractory exacerbation of asthma"                           
[29] "Non-IgE mediated allergic asthma"                                         
[30] "Intermittent allergic asthma"                                             
[31] "Intermittent asthma co-occurrent with allergic rhinitis"                  
[32] "Asthma-chronic obstructive pulmonary disease overlap syndrome"            
[33] "Brittle asthma"                                                           
[34] "Mild persistent allergic asthma"                                          
[35] "Childhood asthma"                                                         
[36] "Moderate persistent allergic asthma"                                      
[37] "Exacerbation of mild persistent asthma"                                   
[38] "Asthmatic bronchitis"                                                     
[39] "Seasonal asthma"                                                          
[40] "Acute severe exacerbation of mild persistent asthma"                      
[41] "Severe persistent allergic asthma"                                        
[42] "Exacerbation of moderate persistent asthma"                               
[43] "Steroid dependent asthma"                                                 
[44] "Exacerbation of severe persistent asthma"                                 
[45] "Intermittent asthma uncontrolled"                                         
[46] "Uncomplicated mild persistent asthma"                                     
[47] "Allergic asthma due to Dermatophagoides farinae"                          
[48] "Mild asthma"                                                              
[49] "Exercise-induced asthma"                                                  
[50] "Mild persistent asthma co-

In [7]:
# Count unique condition_start_datetime for each person_id
number_condition_entries <- asthma_condition_df %>%
  group_by(person_id) %>%
  summarise(unique_conditions = n_distinct(condition_start_datetime))


In [8]:
# Only those with at least two entries are cases
condition_entries_cases <- number_condition_entries[number_condition_entries$unique_conditions>=2, ]
nrow(condition_entries_cases)

[1] 40537

In [9]:
#Get cases before exclusion criteria
asthma_preliminary_cases <- intersect(asthma_drug_cases, condition_entries_cases$person_id)
length(asthma_preliminary_cases)

[1] 36908

# Case exclusion

In [10]:
library(tidyverse)
library(bigrquery)

# This query represents dataset "asthma_case_exclusion_1" for domain "condition" and was generated for All of Us Controlled Tier Dataset v7
dataset_22769885_condition_sql <- paste("
    SELECT
        c_occurrence.person_id,
        c_occurrence.condition_concept_id,
        c_standard_concept.concept_name as standard_concept_name,
        c_standard_concept.concept_code as standard_concept_code,
        c_standard_concept.vocabulary_id as standard_vocabulary,
        c_occurrence.condition_start_datetime 
    FROM
        ( SELECT
            * 
        FROM
            `condition_occurrence` c_occurrence 
        WHERE
            (
                condition_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `cb_criteria` cr       
                    WHERE
                        concept_id IN (192275, 255573, 261325, 315831, 4046868, 4151240, 4195694, 4284110, 4309320, 435514, 441267, 444084, 80809)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1) 
                OR  condition_source_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `cb_criteria` cr       
                    WHERE
                        concept_id IN (44828100, 44830110)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 0 
                    AND is_selectable = 1)
            )  
            AND (
                c_occurrence.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `cb_search_person` p 
                    WHERE
                        has_ehr_data = 1 ) )
            )) c_occurrence 
    LEFT JOIN
        `concept` c_standard_concept 
            ON c_occurrence.condition_concept_id = c_standard_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
condition_22769885_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "condition_22769885",
  "condition_22769885_*.csv")
message(str_glue('The data will be written to {condition_22769885_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_22769885_condition_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  condition_22769885_path,
  destination_format = "CSV")


# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {condition_22769885_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
case_exclusion_1 <- read_bq_export_from_workspace_bucket(condition_22769885_path)

unique(case_exclusion_1$standard_concept_name)

The data will be written to gs://fc-secure-672eeb92-4859-4ed9-9f59-d4349f3534a0/bq_exports/micah_hysong@researchallofus.org/20250825/condition_22769885/condition_22769885_*.csv. Use this path when reading the data into your notebooks in the future.

Loading gs://fc-secure-672eeb92-4859-4ed9-9f59-d4349f3534a0/bq_exports/micah_hysong@researchallofus.org/20250825/condition_22769885/condition_22769885_000000000000.csv.



[1] "Rheumatoid arthritis of left hand"                                           
  [2] "Rheumatoid arthritis of right shoulder"                                      
  [3] "Rheumatoid aortitis"                                                         
  [4] "Maple-bark strippers' lung"                                                  
  [5] "Pulmonary insufficiency following surgery"                                   
  [6] "Compensatory emphysema"                                                      
  [7] "Rheumatoid arthritis of elbow"                                               
  [8] "Transplant rejection of cornea of left eye"                                  
  [9] "Juvenile seropositive polyarthritis"                                         
 [10] "Diabetes mellitus associated with cystic fibrosis"                           
 [11] "Rheumatoid arthritis of left hip"                                            
 [12] "Vocal cord dysfunction"                                                      
 [13] "Bird-fanciers' lung"                                                         
 [14] "Rheumatoid arteritis"                                                        
 [15] "Acute exacerbation of chronic obstructive airways disease with asthma"       
 [16] "Chronic bullous emphysema"                                                   
 [17] "Chronic obstructive lung disease co-occurrent with acute bronchitis"         
 [18] "Chronic obliterative bronchiolitis"                                          
 [19] "Rheumatoid arthritis - hand joint"                                           
 [20] "Transplant failure of cornea of right eye"                                   
 [21] "Rheumatoid arthritis of cervical spine"                                      
 [22] "Acute rejection of lung transplant"                                          
 [23] "Bone marrow transplant rejection"                                            
 [24] "Chronic rejection of renal transplant"                                       
 [25] "Seropositive rheumatoid arthritis of multiple joints"                        
 [26] "Subcutaneous rheumatoid nodule"                                              
 [27] "Bilateral rheumatoid arthritis of knees"                                     
 [28] "Sick building syndrome"                                                      
 [29] "Centriacinar emphysema"                                                      
 [30] "Lymphoproliferative disorder following transplantation"                      
 [31] "Suberosis"                                                                   
 [32] "Transplant failure of cornea of left eye"                                    
 [33] "Rheumatoid osteoperiostitis"                                                 
 [34] "Acute rejection of liver transplant"                                         
 [35] "Rheumatoid arthritis of distal radioulnar joint"                             
 [36] "Rheumatoid arthritis of shoulder"                                            
 [37] "Rheumatoid arthritis of right knee"                                          
 [38] "Cardiac transplant failure"                                                  
 [39] "Interstitial emphysema of lung"                                              
 [40] "Rheumatoid arthritis of right foot"                                          
 [41] "Cystic fibrosis without meconium ileus"                                      
 [42] "Farmers' lung"                                                               
 [43] "Chronic rejection of liver transplant"                                       
 [44] "Acute rejection of pancreas transplant"                                      
 [45] "Bagassosis"                                                                  
 [46] "Rheumatoid arthritis of wrist"                                               
 [47] "Bone marrow transplant failure"                                              
 [48] "

In [11]:
library(tidyverse)
library(bigrquery)

# This query represents dataset "asthma_case_exclusion_1" for domain "procedure" and was generated for All of Us Controlled Tier Dataset v7
dataset_15254378_procedure_sql <- paste("
    SELECT
        procedure.person_id,
        procedure.procedure_concept_id,
        p_standard_concept.concept_name as standard_concept_name,
        p_standard_concept.concept_code as standard_concept_code,
        p_standard_concept.vocabulary_id as standard_vocabulary,
        procedure.procedure_datetime 
    FROM
        ( SELECT
            * 
        FROM
            `procedure_occurrence` procedure 
        WHERE
            (
                procedure_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `cb_criteria` cr       
                    WHERE
                        concept_id IN (4208341)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 1 
                    AND is_selectable = 1)
            )  
            AND (
                procedure.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `cb_search_person` p 
                    WHERE
                        has_ehr_data = 1 ) )
            )) procedure 
    LEFT JOIN
        `concept` p_standard_concept 
            ON procedure.procedure_concept_id = p_standard_concept.concept_id", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
procedure_15254378_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "procedure_15254378",
  "procedure_15254378_*.csv")
message(str_glue('The data will be written to {procedure_15254378_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_15254378_procedure_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  procedure_15254378_path,
  destination_format = "CSV")


# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {procedure_15254378_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(standard_concept_name = col_character(), standard_concept_code = col_character(), standard_vocabulary = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
case_exclusion_2 <- read_bq_export_from_workspace_bucket(procedure_15254378_path)

unique(case_exclusion_2$standard_concept_name)

The data will be written to gs://fc-secure-672eeb92-4859-4ed9-9f59-d4349f3534a0/bq_exports/micah_hysong@researchallofus.org/20250825/procedure_15254378/procedure_15254378_*.csv. Use this path when reading the data into your notebooks in the future.

Loading gs://fc-secure-672eeb92-4859-4ed9-9f59-d4349f3534a0/bq_exports/micah_hysong@researchallofus.org/20250825/procedure_15254378/procedure_15254378_000000000000.csv.



[1] "Donor renal transplantation"   "Orthotopic liver transplant"  
 [3] "Cadaveric renal transplant"    "Autotransplantation of kidney"
 [5] "Transplantation of pancreas"   "Transplantation of heart"     
 [7] "Single lung transplant"        "Double lung transplant"       
 [9] "Homotransplant of pancreas"    "Transplantation of liver"     
[11] "Transplant of lung"            "Live donor renal transplant"  
[13] "Transplant of kidney"          "Heterotopic liver transplant"

In [12]:
asthma_case_exclusion <- unique(c(case_exclusion_1$person_id, case_exclusion_2$person_id))
length(asthma_case_exclusion)

[1] 36228

In [13]:
asthma_cases<-setdiff(asthma_preliminary_cases, asthma_case_exclusion)
length(asthma_cases)

[1] 25296

# Get Controls

In [14]:
# This snippet assumes that you run setup first

# This code copies a file from your Google Bucket into a dataframe

# replace 'test.csv' with the name of the file in your google bucket (don't delete the quotation marks)
name_of_file_in_bucket <- 'Demographic_and_ancestry_covariates.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ", my_bucket, "/data/", name_of_file_in_bucket, " ."), intern=T)

# Load the file into a dataframe
demographics  <- read_csv(name_of_file_in_bucket)

character(0)

Rows: 162193 Columns: 28
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (8): SexGender, income, education, where_born, military, healthcare, di...
dbl  (9): person_id, race_unknown, age_today, LGBTQIA, ehr_length, relative_...
lgl  (8): AIAN, Asian, Black, Mid, Multiple, PI, White, His
date (3): date_of_birth, min_date, max_date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:
library(tidyverse)
library(bigrquery)

# This query represents dataset "asthma_survey" for domain "survey" and was generated for All of Us Controlled Tier Dataset v8
dataset_39470085_survey_sql <- paste("
    SELECT
        answer.person_id,
        answer.survey_datetime,
        answer.survey,
        answer.question_concept_id,
        answer.question,
        answer.answer_concept_id,
        answer.answer,
        answer.survey_version_concept_id,
        answer.survey_version_name  
    FROM
        `ds_survey` answer   
    WHERE
        (
            question_concept_id IN (836815)
        )  
        AND (
            answer.PERSON_ID IN (SELECT
                distinct person_id  
            FROM
                `cb_search_person` cb_search_person  
            WHERE
                cb_search_person.person_id IN (SELECT
                    person_id 
                FROM
                    `cb_search_person` p 
                WHERE
                    has_whole_genome_variant = 1 ) 
                AND cb_search_person.person_id IN (SELECT
                    person_id 
                FROM
                    `cb_search_person` p 
                WHERE
                    has_ehr_data = 1 ) )
        )", sep="")

# Formulate a Cloud Storage destination path for the data exported from BigQuery.
# NOTE: By default data exported multiple times on the same day will overwrite older copies.
#       But data exported on a different days will write to a new location so that historical
#       copies can be kept as the dataset definition is changed.
survey_39470085_path <- file.path(
  Sys.getenv("WORKSPACE_BUCKET"),
  "bq_exports",
  Sys.getenv("OWNER_EMAIL"),
  strftime(lubridate::now(), "%Y%m%d"),  # Comment out this line if you want the export to always overwrite.
  "survey_39470085",
  "survey_39470085_*.csv")
message(str_glue('The data will be written to {survey_39470085_path}. Use this path when reading ',
                 'the data into your notebooks in the future.'))

# Perform the query and export the dataset to Cloud Storage as CSV files.
# NOTE: You only need to run `bq_table_save` once. After that, you can
#       just read data from the CSVs in Cloud Storage.
bq_table_save(
  bq_dataset_query(Sys.getenv("WORKSPACE_CDR"), dataset_39470085_survey_sql, billing = Sys.getenv("GOOGLE_PROJECT")),
  survey_39470085_path,
  destination_format = "CSV")


# Read the data directly from Cloud Storage into memory.
# NOTE: Alternatively you can `gsutil -m cp {survey_39470085_path}` to copy these files
#       to the Jupyter disk.
read_bq_export_from_workspace_bucket <- function(export_path) {
  col_types <- cols(survey = col_character(), question = col_character(), answer = col_character(), survey_version_name = col_character())
  bind_rows(
    map(system2('gsutil', args = c('ls', export_path), stdout = TRUE, stderr = TRUE),
        function(csv) {
          message(str_glue('Loading {csv}.'))
          chunk <- read_csv(pipe(str_glue('gsutil cat {csv}')), col_types = col_types, show_col_types = FALSE)
          if (is.null(col_types)) {
            col_types <- spec(chunk)
          }
          chunk
        }))
}
survey_df <- read_bq_export_from_workspace_bucket(survey_39470085_path)

unique(survey_df$answer)


The data will be written to gs://fc-secure-672eeb92-4859-4ed9-9f59-d4349f3534a0/bq_exports/micah_hysong@researchallofus.org/20250825/survey_39470085/survey_39470085_*.csv. Use this path when reading the data into your notebooks in the future.



In [ ]:
asthma_survey<-survey_df[survey_df$answer == "Including yourself, who in your family has had asthma? - Self",]
asthma_survey<-unique(asthma_survey$person_id)
length(asthma_survey)

In [ ]:
exclude<- unique(c(number_condition_entries$person_id, asthma_drug_cases, asthma_case_exclusion, asthma_survey))
length(exclude)

In [ ]:
controls<-setdiff(demographics$person_id, exclude)
length(controls)

# Make Case Control DF

In [ ]:
df_cases <- data.frame(
  person_id = asthma_cases,
  status = 1
)

df_controls <- data.frame(
  person_id = controls,
  status = 0
)

final_df <- rbind(df_cases, df_controls)
nrow(final_df)
n_distinct(final_df$person_id)

In [ ]:
# This snippet assumes that you run setup first

# This code saves your dataframe into a csv file in a "data" folder in Google Bucket

# Replace df with THE NAME OF YOUR DATAFRAME
my_dataframe <- final_df

# Replace 'test.csv' with THE NAME of the file you're going to store in the bucket (don't delete the quotation marks)
destination_filename <- 'eMERGE_asthma_case_control_df.csv'

########################################################################
##
################# DON'T CHANGE FROM HERE ###############################
##
########################################################################

# store the dataframe in current workspace
write_excel_csv(my_dataframe, destination_filename)

# Get the bucket name
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# Copy the file from current workspace to the bucket
system(paste0("gsutil cp ./", destination_filename, " ", my_bucket, "/data/"), intern=T)

# Check if file is in the bucket
system(paste0("gsutil ls ", my_bucket, "/data/*.csv"), intern=T)
